# Gender recognition by voice

In this project, I am going to train a model which can take voices as inputs and recognize if the voice is a male voice or a female voice. 

For training the model, I am using a dataset which contains the following attributes:

**meanfreq:** mean frequency (in kHz)

**sd:** standard deviation of frequency

**median:**** median frequency (in kHz)

**Q25:** first quantile (in kHz)

**Q75:** third quantile (in kHz)

**IQR:** interquantile range (in kHz)

**skew:** skewness (see note in specprop description)

**kurt:** kurtosis (see note in specprop description)

**sp.ent:** spectral entropy

**sfm:** spectral flatness

**mode:** mode frequency

**centroid:** frequency centroid (see specprop)

**peakf:** peak frequency (frequency with highest energy)

**meanfun:** average of fundamental frequency measured across acoustic signal

**minfun:** minimum fundamental frequency measured across acoustic signal

**maxfun:** maximum fundamental frequency measured across acoustic signal

**meandom:** average of dominant frequency measured across acoustic signal

**mindom:** minimum of dominant frequency measured across acoustic signal

**maxdom:** maximum of dominant frequency measured across acoustic signal

**dfrange:** range of dominant frequency measured across acoustic signal

**modindx:** modulation index. Calculated as the accumulated absolute difference between adjacent measurements of fundamental frequencies divided by the frequency range

**label:** male or female

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.naive_bayes import GaussianNB
import seaborn as sns
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
voice_df = pd.read_csv('voice.csv')
voice_df.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [3]:
voice_df.isnull().sum()

meanfreq    0
sd          0
median      0
Q25         0
Q75         0
IQR         0
skew        0
kurt        0
sp.ent      0
sfm         0
mode        0
centroid    0
meanfun     0
minfun      0
maxfun      0
meandom     0
mindom      0
maxdom      0
dfrange     0
modindx     0
label       0
dtype: int64

In [4]:
voice_df['label'] = voice_df['label'].map({'male':0, 'female':1})

In [5]:
voice_df.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,0
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,0
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,0
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,0
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,0


In [6]:
print(voice_df.columns)

Index(['meanfreq', 'sd', 'median', 'Q25', 'Q75', 'IQR', 'skew', 'kurt',
       'sp.ent', 'sfm', 'mode', 'centroid', 'meanfun', 'minfun', 'maxfun',
       'meandom', 'mindom', 'maxdom', 'dfrange', 'modindx', 'label'],
      dtype='object')


In [7]:
X = voice_df.drop('label', axis = 1)
y = voice_df['label']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Machine Learning Models

### Logistic Regression

In [10]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)
logistic_pred = logistic_model.predict(X_test)

In [11]:
logistic_accuracy = accuracy_score(y_test, logistic_pred)
logistic_confusion_mat = confusion_matrix(y_test, logistic_pred)
logistic_classification_rep = classification_report(y_test, logistic_pred)

In [12]:
print("Logistic Regression Results:")
print(f'Accuracy: {logistic_accuracy}')
print(f'Confusion Matrix:\n{logistic_confusion_mat}')
print(f'Classification Report:\n{logistic_classification_rep}')

Logistic Regression Results:
Accuracy: 0.9810725552050473
Confusion Matrix:
[[330   7]
 [  5 292]]
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       337
           1       0.98      0.98      0.98       297

    accuracy                           0.98       634
   macro avg       0.98      0.98      0.98       634
weighted avg       0.98      0.98      0.98       634



### Naive Bayes

In [13]:
naive_bayes_model = GaussianNB()
naive_bayes_model.fit(X_train, y_train)
naive_bayes_pred = naive_bayes_model.predict(X_test)

In [14]:
naive_bayes_accuracy = accuracy_score(y_test, naive_bayes_pred)
naive_bayes_confusion_mat = confusion_matrix(y_test, naive_bayes_pred)
naive_bayes_classification_rep = classification_report(y_test, naive_bayes_pred)

In [15]:
print("\nNaive Bayes Results:")
print(f'Accuracy: {naive_bayes_accuracy}')
print(f'Confusion Matrix:\n{naive_bayes_confusion_mat}')
print(f'Classification Report:\n{naive_bayes_classification_rep}')


Naive Bayes Results:
Accuracy: 0.9085173501577287
Confusion Matrix:
[[306  31]
 [ 27 270]]
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.91      0.91       337
           1       0.90      0.91      0.90       297

    accuracy                           0.91       634
   macro avg       0.91      0.91      0.91       634
weighted avg       0.91      0.91      0.91       634



## Deep Learning Models

### Artifical Neural Network

In [16]:
model = Sequential()
model.add(Dense(units=128, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [17]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10


80/80 [==============================] - 2s 4ms/step - loss: 0.2291 - accuracy: 0.9345 - val_loss: 0.0818 - val_accuracy: 0.9779
Epoch 2/10
80/80 [==============================] - 0s 1ms/step - loss: 0.0791 - accuracy: 0.9751 - val_loss: 0.0672 - val_accuracy: 0.9795
Epoch 3/10
80/80 [==============================] - 0s 1ms/step - loss: 0.0663 - accuracy: 0.9779 - val_loss: 0.0629 - val_accuracy: 0.9811
Epoch 4/10
80/80 [==============================] - 0s 1ms/step - loss: 0.0585 - accuracy: 0.9795 - val_loss: 0.0583 - val_accuracy: 0.9826
Epoch 5/10
80/80 [==============================] - 0s 1ms/step - loss: 0.0518 - accuracy: 0.9830 - val_loss: 0.0719 - val_accuracy: 0.9779
Epoch 6/10
80/80 [==============================] - 0s 1ms/step - loss: 0.0500 - accuracy: 0.9834 - val_loss: 0.0667 - val_accuracy: 0.9779
Epoch 7/10
80/80 [==============================] - 0s 1ms/step - loss: 0.0459 - accuracy: 0.9830 - val_loss: 0.0649 - val_accuracy: 0.9795
Epoch 8/10
80/80 

In [18]:
_, nn_accuracy = model.evaluate(X_test, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

20/20 [==============================] - 0s 945us/step - loss: 0.0562 - accuracy: 0.9811
Neural Network Accuracy: 0.9810725450515747
